In [0]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [0]:
# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [0]:
#I'm using Beautifyl Soup to scrape Wikipedia
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

res = requests.get(URL).text
soup = BeautifulSoup(res,'lxml')
for items in soup.find('table', class_='wikitable sortable').find_all('tr')[1::1]:
    data = items.find_all(['th','td'])
    try:
        PostalCode = data[0].text
        Borough = data[1].text
        Neighborhood = data[2].text.split("\n")[0]       
    except: pass
    neighborhoods = neighborhoods.append({'Borough': Borough,
                                          'Neighborhood': Neighborhood,
                                          'PostalCode': PostalCode}, ignore_index=True)

In [122]:
#Here is the input pandas data frame from Wikipedia
neighborhoods.head(20)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [0]:
# Remove unassigned neighborhoods
neighborhoods = neighborhoods[neighborhoods.Borough != 'Not assigned']

In [115]:
neighborhoods.head(15)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [0]:
# Concatonate neighborhoods in same postal code
grouped = neighborhoods.groupby('PostalCode').agg(lambda x: ','.join(set(x)))

In [118]:
grouped.head()

,Borough,Neighborhood
PostalCode,,
M1B,Scarborough,"Malvern,Rouge"
M1C,Scarborough,"Highland Creek,Port Union,Rouge Hill"
M1E,Scarborough,"West Hill,Morningside,Guildwood"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [0]:
# This finds all still unassigned neighborhoods and gives them the borough name
mask = grouped.Neighborhood == 'Not assigned'

grouped.loc[mask,'Neighborhood'] = grouped.loc[mask,'Borough']

In [125]:
grouped.loc[mask,:]

,Borough,Neighborhood
PostalCode,,
M7A,Queen's Park,Queen's Park


In [126]:
#Here is the shape of my output dataframe
grouped.shape

(103, 2)

In [127]:
grouped

,Borough,Neighborhood
PostalCode,,
M1B,Scarborough,"Malvern,Rouge"
M1C,Scarborough,"Highland Creek,Port Union,Rouge Hill"
M1E,Scarborough,"West Hill,Morningside,Guildwood"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
M1J,Scarborough,Scarborough Village
M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
M1M,Scarborough,"Cliffside,Cliffcrest,Scarborough Village West"
